In [26]:
import pandas as pd
import sqlite3

In [27]:
with sqlite3.connect('NDT/ndt_sql.db') as con:
    ndt.to_sql("ndt", con)

In [59]:
def pdquery(db, sql, params=()):
    with sqlite3.connect(db) as con:
        res = pd.read_sql_query(sql, con, params = params)
    return res

def query(db, sql, params=()):
    with sqlite3.connect(db) as con:
        res = con.execute(sql,params)
    return res.fetchall()

In [36]:
ndt_db = 'NDT/ndt_sql.db'

In [89]:
df = pdquery(ndt_db, """
    select 
        a.sent_id, 
        a.token_order as dep_id, 
        b.token_order as head_id, 
        a.form,  
        a.deprel, 
        b.form,
        a.lemma as lemma_dep, 
        b.lemma as lemma_head
    from 
        ndt as a, 
        ndt as b 
    where 
        a.deprel = 'KOORD'  
        and a.head = b.token_order 
        and a.sent_id = b.sent_id 
    """)

In [91]:
df['diff'] = df.dep_id - df.head_id

In [93]:
df[abs(df['diff']) == 2]

,sent_id,dep_id,head_id,form,deprel,form,lemma_dep,lemma_head,diff
0,1,3,1,piggvar,KOORD,Lam,piggvar,lam,2
1,2,3,1,piggvar,KOORD,Kamskjell,piggvar,kamskjell,2
3,4,24,22,ruccolasalat,KOORD,trøffelhonningvinaigrette,ruccolasalat,trøffelhonningvinaigrette,2
5,6,10,8,blomkålpuré,KOORD,klasetomatbåter,blomkålpuré,klasetomatbåt,2
7,6,22,20,fenikkelsmak,KOORD,appelsin,fenikkelsmak,appelsin,2
...,...,...,...,...,...,...,...,...,...
22949,37610,10,8,trekkvegar,KOORD,bruk,trekkveg,bruk,2
22951,37610,23,21,vinterbeite,KOORD,sommar-,vinterbeite,sommar-,2
22954,37614,23,21,strukturteljingar,KOORD,kalve-,strukturteljing,kalve-,2
22956,37617,8,6,å,KOORD,stamme,å,stamme,2


In [95]:
df['diff'].describe()

count    22963.000000
mean         6.041066
std          5.476702
min          1.000000
25%          2.000000
50%          4.000000
75%          8.000000
max         76.000000
Name: diff, dtype: float64

In [86]:
def make_sentence(identifier):
    return ' '.join([x[0] for x in query(ndt_db, "select form from ndt where sent_id = ? order by rowid", (identifier,))])

In [87]:
make_sentence(5)

'Deretter fulgte " Turbot au Four - purée de chou-fleur " .'

In [4]:
ndt = pd.read_csv("NDT/ndt_all_reduce.csv", index_col = 0)

In [25]:
for i,x in enumerate(ndt[ndt["deprel"] == 'KOORD'][[
    'token_id', 'sent_id', 'head', 'form', 'lemma'
]].iterrows()):
    
    if i > 10:
        break
    
    print(ndt[(ndt.sent_id == x[1]['sent_id']) & (ndt.token_id == x[1]['head'])])

   token_id  token_order form lemma    pos               feats  head deprel  \
0         1            1  Lam   lam  subst  appell|nøyt|ub|ent     0   FRAG   

   sent_id  ud_id language_code data_split  newpar  document_id  
0        1      1         nb-NO      train       0          NaN  
Empty DataFrame
Columns: [token_id, token_order, form, lemma, pos, feats, head, deprel, sent_id, ud_id, language_code, data_split, newpar, document_id]
Index: []
Empty DataFrame
Columns: [token_id, token_order, form, lemma, pos, feats, head, deprel, sent_id, ud_id, language_code, data_split, newpar, document_id]
Index: []
Empty DataFrame
Columns: [token_id, token_order, form, lemma, pos, feats, head, deprel, sent_id, ud_id, language_code, data_split, newpar, document_id]
Index: []
Empty DataFrame
Columns: [token_id, token_order, form, lemma, pos, feats, head, deprel, sent_id, ud_id, language_code, data_split, newpar, document_id]
Index: []
Empty DataFrame
Columns: [token_id, token_order, form, lemma,